In [1]:
import pandas as pd
import os
import re
import numpy as np

# ---------- paths ----------
hosp_path = r"C:\Users\Coditas\Desktop\Projects\Cauti\raw\hosp"
icu_path = r"C:\Users\Coditas\Desktop\Projects\Cauti\raw\icu"
output_path = os.path.join(hosp_path, "output")
os.makedirs(output_path, exist_ok=True)

# ---------- load files ----------
patients = pd.read_csv(os.path.join(hosp_path, "patients.csv"))
admissions = pd.read_csv(os.path.join(hosp_path, "admissions.csv"))
diagnoses_icd = pd.read_csv(os.path.join(hosp_path, "diagnoses_icd.csv"))
d_icd = pd.read_csv(os.path.join(hosp_path, "d_icd_diagnoses.csv"))

# ---------- ICD lists ----------
icd9_cauti = ["99664"]
icd10_cauti = ["T83511A","T83511D","T83511S","T83518A","T83518D","T83518S","T8351XA"]

icd9_uti = [
    "5950","5951","5952","5953","5954","5959",
    "5900","5901","5902","5903","5908","5909",
    "5990","59780","59789"
]

icd10_uti = [
    "N300","N301","N302","N303","N304","N308","N309",
    "N10","N11","N110","N119","N12",
    "N341","N342","N349","N390"
]

# ---------- helpers ----------
def normalize_icd(code):
    if pd.isna(code):
        return None
    s = str(code).strip().upper()
    s = s.replace(".", "").replace(" ", "")
    s = re.sub(r"[^A-Z0-9]", "", s)
    return s if s else None

diagnoses_icd["icd_code_norm"] = diagnoses_icd["icd_code"].apply(normalize_icd)
diagnoses_icd["icd_version"] = pd.to_numeric(
    diagnoses_icd.get("icd_version", pd.Series()),
    errors="coerce"
).astype("Int64")

d_icd["icd_code_norm"] = d_icd["icd_code"].apply(normalize_icd)
d_icd["icd_version"] = pd.to_numeric(
    d_icd.get("icd_version", pd.Series()),
    errors="coerce"
).astype("Int64")

def build_contains_mask(df, candidates, version):
    mask = pd.Series(False, index=df.index)
    for c in candidates:
        mask |= df["icd_code_norm"].astype(str).str.startswith(c, na=False)
    return (df["icd_version"] == version) & mask

# ---------- normalize ICD lists ----------
icd9_cauti_norm = [normalize_icd(x) for x in icd9_cauti]
icd10_cauti_norm = [normalize_icd(x) for x in icd10_cauti]
icd9_uti_norm = [normalize_icd(x) for x in icd9_uti]
icd10_uti_norm = [normalize_icd(x) for x in icd10_uti]

# ---------- identify CAUTI / UTI ----------
mask_strict = (
    build_contains_mask(diagnoses_icd, icd9_cauti_norm, 9) |
    build_contains_mask(diagnoses_icd, icd10_cauti_norm, 10)
)
strict_dx = diagnoses_icd[mask_strict].copy()
strict_dx["cauti_type"] = "CAUTI"

mask_prob = (
    build_contains_mask(diagnoses_icd, icd9_uti_norm, 9) |
    build_contains_mask(diagnoses_icd, icd10_uti_norm, 10)
)
uti_dx = diagnoses_icd[mask_prob].copy()
uti_dx["cauti_type"] = "Probable_CAUTI"

combined = pd.concat([strict_dx, uti_dx], ignore_index=True).drop_duplicates(
    subset=["subject_id","hadm_id","icd_code_norm","cauti_type"]
)

# ---------- ICD dictionary ----------
d_icd_subset = d_icd[["icd_code_norm","icd_version","long_title"]].drop_duplicates()
dict_map = {
    (str(r.icd_code_norm), int(r.icd_version) if pd.notna(r.icd_version) else None): r.long_title
    for _, r in d_icd_subset.iterrows()
}

# ---------- ICD aggregation ----------
v9 = diagnoses_icd[diagnoses_icd["icd_version"] == 9] \
    .groupby(["subject_id","hadm_id"], as_index=False)["icd_code_norm"] \
    .agg(list).rename(columns={"icd_code_norm":"icd_version_9"})

v10 = diagnoses_icd[diagnoses_icd["icd_version"] == 10] \
    .groupby(["subject_id","hadm_id"], as_index=False)["icd_code_norm"] \
    .agg(list).rename(columns={"icd_code_norm":"icd_version_10"})

icd_all = pd.merge(v9, v10, on=["subject_id","hadm_id"], how="outer")

def row_to_icd_dict(r):
    out = {}
    if isinstance(r.get("icd_version_9"), list) and r["icd_version_9"]:
        out["icd_version_9"] = r["icd_version_9"]
    if isinstance(r.get("icd_version_10"), list) and r["icd_version_10"]:
        out["icd_version_10"] = r["icd_version_10"]
    return out

icd_all["icd_codes"] = icd_all.apply(row_to_icd_dict, axis=1)
icd_all = icd_all[["subject_id","hadm_id","icd_codes"]]

# ---------- CAUTI aggregation ----------
def aggregate_cauti_rows(df):
    cauti = df[df["cauti_type"]=="CAUTI"][["icd_code_norm","icd_version"]]
    uti = df[df["cauti_type"]=="Probable_CAUTI"][["icd_code_norm","icd_version"]]

    cauti_dict = {
        r.icd_code_norm: dict_map.get((r.icd_code_norm, int(r.icd_version)), None)
        for _, r in cauti.drop_duplicates().iterrows()
    }
    uti_dict = {
        r.icd_code_norm: dict_map.get((r.icd_code_norm, int(r.icd_version)), None)
        for _, r in uti.drop_duplicates().iterrows()
    }

    if cauti_dict and uti_dict:
        ctype = "CAUTI+OtherUTI"
    elif cauti_dict:
        ctype = "CAUTI"
    elif uti_dict:
        ctype = "Probable_CAUTI"
    else:
        ctype = "No_CAUTI"

    return pd.Series({
        "cauti_type": ctype,
        "cauti_icd_codes": cauti_dict,
        "other_uti_icd_codes": uti_dict
    })

cauti_agg = (
    combined
    .groupby(["subject_id","hadm_id"], as_index=False)
    .apply(aggregate_cauti_rows, include_groups=False)
)

# ---------- base admission ----------
cauti_per_admission_full = admissions[["subject_id","hadm_id"]].drop_duplicates() \
    .merge(cauti_agg, on=["subject_id","hadm_id"], how="left")

cauti_per_admission_full["cauti_type"] = cauti_per_admission_full["cauti_type"].fillna("No_CAUTI")
cauti_per_admission_full["cauti_icd_codes"] = cauti_per_admission_full["cauti_icd_codes"].apply(lambda x: x if isinstance(x, dict) else {})
cauti_per_admission_full["other_uti_icd_codes"] = cauti_per_admission_full["other_uti_icd_codes"].apply(lambda x: x if isinstance(x, dict) else {})

# ---------- merge ICD codes ----------
cauti_per_admission_full = cauti_per_admission_full.merge(
    icd_all, on=["subject_id","hadm_id"], how="left"
)

# 🔧 CRITICAL FIX — force icd_codes to dict
cauti_per_admission_full["icd_codes"] = cauti_per_admission_full["icd_codes"].apply(
    lambda x: x if isinstance(x, dict) else {}
)

# ---------- remaining ICDs ----------
cauti_per_admission_full["remaining_icd_codes"] = cauti_per_admission_full.apply(
    lambda r: {
        code: dict_map.get((code, 9 if k=="icd_version_9" else 10))
        for k, codes in r.icd_codes.items()
        for code in codes
        if code not in r.cauti_icd_codes and code not in r.other_uti_icd_codes
    },
    axis=1
)

cauti_per_admission_full.drop(columns=["icd_codes"], inplace=True)

# ---------- demographics ----------
cauti_per_admission_full = cauti_per_admission_full.merge(
    patients[["subject_id","gender","anchor_age"]].drop_duplicates(),
    on="subject_id",
    how="left"
)

# ---------- admission details ----------
cauti_per_admission_full = cauti_per_admission_full.merge(
    admissions[
        ["subject_id","hadm_id","admittime","dischtime",
         "admission_type","admission_location",
         "discharge_location","race"]
    ].drop_duplicates(),
    on=["subject_id","hadm_id"],
    how="left"
)

   subject_id   hadm_id cauti_type cauti_icd_codes other_uti_icd_codes  \
0    10000032  22595853   No_CAUTI              {}                  {}   
1    10000032  25742920   No_CAUTI              {}                  {}   
2    10000032  29079034   No_CAUTI              {}                  {}   
3    10000560  28979390   No_CAUTI              {}                  {}   
4    10000690  25860671   No_CAUTI              {}                  {}   

                                 remaining_icd_codes gender  anchor_age  \
0  {'5723': 'Portal hypertension', '78959': 'Othe...      F          52   
1  {'07054': 'Chronic hepatitis C without mention...      F          52   
2  {'45829': 'Other iatrogenic hypotension', '070...      F          52   
3  {'1890': 'Malignant neoplasm of kidney, except...      F          53   
4  {'5070': 'Pneumonitis due to inhalation of foo...      F          86   

             admittime            dischtime               admission_type  \
0  2180-05-06 22:23:00  2180

In [2]:
cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race'],
      dtype='object')

## Add Procedures

In [4]:
# -----------------------------
# Add catheter INSERTION / REMOVAL procedure columns (SAFE)
# -----------------------------
d_icd_procedures = pd.read_csv(
    os.path.join(hosp_path, "d_icd_procedures.csv"),
    dtype=str
)

procedures = pd.read_csv(
    os.path.join(hosp_path, "procedures_icd.csv"),
    dtype=str,
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version", "chartdate"]
)

procedures["icd_version"] = pd.to_numeric(
    procedures["icd_version"], errors="coerce"
).astype("Int64")
procedures["chartdate"] = pd.to_datetime(
    procedures["chartdate"], errors="coerce"
)
procedures["icd_code"] = procedures["icd_code"].astype(str)

# --------------------------------------------------
# Broad detection + strict insertion/removal sets
# --------------------------------------------------
icd9_catheter_codes = {"598","5994","9646","9647","9648","9762","9763","5794"}
icd10_catheter_prefixes = ["0T9B","0T9C","0T2B","0TPB","0TRB","0TWB"]

insertion_icd9 = {"598", "5994", "5794"}
insertion_icd10_prefixes = ("0T9B", "0T9C", "0T2B")

removal_icd9 = {"9762", "9763"}
removal_icd10_prefixes = ("0TPB",)

# --------------------------------------------------
# Identify catheter-related procedure rows
# --------------------------------------------------
def is_catheter_proc_row(r):
    code = str(r["icd_code"])
    ver = r["icd_version"]
    if pd.isna(ver):
        return False
    ver = int(ver)
    if ver == 9:
        return code in icd9_catheter_codes
    if ver == 10:
        return any(code.startswith(p) for p in icd10_catheter_prefixes)
    return False

procedures["is_catheter"] = procedures.apply(
    is_catheter_proc_row, axis=1
)

cat_procs = procedures[procedures["is_catheter"]].copy()

# --------------------------------------------------
# Catheter type classifier
# --------------------------------------------------
def classify_catheter_type(code):
    code = str(code)
    if code.startswith("0T9B"):
        return "foley"
    if code.startswith("0T2B"):
        return "suprapubic"
    if code.startswith("0T9C"):
        return "ureteral"
    if code in {"9648", "5794"}:
        return "foley"
    if code in {"5994", "9647", "9763"}:
        return "suprapubic"
    if code in {"598", "9646", "9762"}:
        return "ureteral"
    return "unknown"

# attach long titles
d_icd_procedures["icd_version"] = pd.to_numeric(
    d_icd_procedures["icd_version"], errors="coerce"
).astype("Int64")

cat_procs = cat_procs.merge(
    d_icd_procedures[["icd_code","icd_version","long_title"]],
    on=["icd_code","icd_version"],
    how="left"
)

# --------------------------------------------------
# Build INSERTION / REMOVAL admission-level table
# --------------------------------------------------
rows = []

for (subj, hadm), grp in cat_procs.groupby(["subject_id","hadm_id"], sort=False):

    grp = grp.sort_values("chartdate")

    proc_dict = {
        str(c): (t if t else None)
        for c, t in zip(
            grp["icd_code"].astype(str),
            grp["long_title"].fillna("").astype(str)
        )
    }

    types_used = sorted({
        classify_catheter_type(c)
        for c in grp["icd_code"].astype(str)
    })

    insertion_mask = grp.apply(
        lambda r: (
            str(r["icd_code"]) in insertion_icd9 or
            any(str(r["icd_code"]).startswith(p) for p in insertion_icd10_prefixes)
        ),
        axis=1
    )

    removal_mask = grp.apply(
        lambda r: (
            str(r["icd_code"]) in removal_icd9 or
            any(str(r["icd_code"]).startswith(p) for p in removal_icd10_prefixes)
        ),
        axis=1
    )

    rows.append({
        "subject_id": str(subj),
        "hadm_id": str(hadm),
    
        # ✅ BROAD evidence (ANY catheter ICD)
        "catheter_procedures": proc_dict,
    
        # ✅ STRICT evidence (insertion / removal only)
        "catheter_procedures_ir": proc_dict,
    
        "catheter_used_in_procedures_ir": types_used,
        "catheter_insertion_date_ir": grp.loc[insertion_mask, "chartdate"].min(),
        "catheter_removal_date_ir": grp.loc[removal_mask, "chartdate"].min()
    })


catheter_ir_df = pd.DataFrame(rows)

# --------------------------------------------------
# Merge WITHOUT overwriting earlier columns
# --------------------------------------------------
for c in ["subject_id","hadm_id"]:
    cauti_per_admission_full[c] = cauti_per_admission_full[c].astype(str)
    catheter_ir_df[c] = catheter_ir_df[c].astype(str)

cauti_per_admission_full = cauti_per_admission_full.merge(
    catheter_ir_df,
    on=["subject_id","hadm_id"],
    how="left"
)

cauti_per_admission_full["catheter_procedures"] = (
    cauti_per_admission_full["catheter_procedures"]
        .apply(lambda x: x if isinstance(x, dict) else {})
)
# Fill safe defaults
cauti_per_admission_full["catheter_procedures_ir"] = (
    cauti_per_admission_full["catheter_procedures_ir"]
        .apply(lambda x: x if isinstance(x, dict) else {})
)

cauti_per_admission_full["catheter_used_in_procedures_ir"] = (
    cauti_per_admission_full["catheter_used_in_procedures_ir"]
        .apply(lambda x: x if isinstance(x, list) else [])
)

cauti_per_admission_full["catheter_insertion_date_ir"] = pd.to_datetime(
    cauti_per_admission_full["catheter_insertion_date_ir"],
    errors="coerce"
)

cauti_per_admission_full["catheter_removal_date_ir"] = pd.to_datetime(
    cauti_per_admission_full["catheter_removal_date_ir"],
    errors="coerce"
)


In [5]:
cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir'],
      dtype='object')

## Add Outputevents

In [7]:
# -----------------------------------------------------
# Add ICU OUTPUTEVENTS catheter info into cauti_per_admission_full
# (extended list of ITEMIDs)
# -----------------------------------------------------

# 1. Urinary catheter related ITEMIDs + meta (extended)
urinary_catheter_itemids = [
    226559,  # Foley
    226561,  # Condom catheter
    226563,  # Suprapubic catheter
    226567,  # Straight cath
    226566,  # GU urine & irrigant out
    227487,  # Irrigant type
    227488,  # Irrigant volume in
    227489,  # Irrigant / urine volume out
    227701,  # Drainage bag
    226557,  # Right ureteral stent
    226558,  # Left ureteral stent
    226564,  # Right nephrostomy
    226565   # Left nephrostomy
]

catheter_item_meta = {
    226559: {"label": "Foley catheter",                       "catheter_type": "foley"},
    226561: {"label": "Condom catheter",                      "catheter_type": "condom"},
    226563: {"label": "Suprapubic catheter",                  "catheter_type": "suprapubic"},
    226567: {"label": "Straight catheter",                    "catheter_type": "straight"},
    226566: {"label": "GU urine & irrigant out",              "catheter_type": None},
    227487: {"label": "Irrigant type",                        "catheter_type": None},
    227488: {"label": "Irrigant volume in",                   "catheter_type": None},
    227489: {"label": "Irrigant / urine volume out",          "catheter_type": None},
    227701: {"label": "Drainage bag",                         "catheter_type": None},

    # newly added
    226557: {"label": "Right ureteral stent",                 "catheter_type": "ureteral"},
    226558: {"label": "Left ureteral stent",                  "catheter_type": "ureteral"},
    226564: {"label": "Right nephrostomy",                    "catheter_type": "nephrostomy"},
    226565: {"label": "Left nephrostomy",                     "catheter_type": "nephrostomy"},
}

# 2. Load OUTPUTEVENTS (only needed columns)
output = pd.read_csv(
    os.path.join(icu_path, "outputevents.csv"),
    usecols=["subject_id", "hadm_id", "itemid"],
    dtype={"subject_id": str, "hadm_id": str, "itemid": str}  # read as strings to avoid dtype issues
)

# normalize itemid to int where possible (coerce invalid -> NaN)
output["itemid"] = pd.to_numeric(output["itemid"], errors="coerce").astype("Int64")
output = output.dropna(subset=["itemid"])  # drop rows with missing itemid after coercion
output["itemid"] = output["itemid"].astype(int)

# 3. Filter to urinary catheter-related outputevents
output_cath = output[output["itemid"].isin(urinary_catheter_itemids)].copy()

# 4. Group per admission and build dictionaries/arrays (explicit loop to avoid groupby.apply deprecation)
cat_rows = []
if not output_cath.empty:
    grouped = output_cath.groupby(["subject_id", "hadm_id"], sort=False)
    for (subj, hadm), grp in grouped:
        itemids = grp["itemid"].unique().tolist()
        outputevents_dict = {}
        catheter_types_used = set()
        for iid in itemids:
            iid_int = int(iid)
            meta = catheter_item_meta.get(iid_int, {})
            label = meta.get("label", "Unknown")
            ctype = meta.get("catheter_type", None)
            outputevents_dict[iid_int] = label
            if ctype is not None:
                catheter_types_used.add(ctype)
        cat_rows.append({
            "subject_id": str(subj),
            "hadm_id": str(hadm),
            "catheter_outputevents": outputevents_dict,
            "catheter_used_in_output_events": sorted(list(catheter_types_used))
        })

# build DataFrame (guarantee columns even if no rows)
if cat_rows:
    catheter_output_admission_level = pd.DataFrame(cat_rows)
else:
    catheter_output_admission_level = pd.DataFrame(columns=[
        "subject_id", "hadm_id", "catheter_outputevents", "catheter_used_in_output_events"
    ])

# 5. Merge into cauti_per_admission_full (align dtypes first)
# Ensure the key columns exist in cauti_per_admission_full
if "subject_id" not in cauti_per_admission_full.columns:
    cauti_per_admission_full["subject_id"] = ""
if "hadm_id" not in cauti_per_admission_full.columns:
    cauti_per_admission_full["hadm_id"] = ""

# Cast both sides to string for safe merging (preserves values)
cauti_per_admission_full["subject_id"] = cauti_per_admission_full["subject_id"].astype(str)
cauti_per_admission_full["hadm_id"] = cauti_per_admission_full["hadm_id"].astype(str)
catheter_output_admission_level["subject_id"] = catheter_output_admission_level["subject_id"].astype(str)
catheter_output_admission_level["hadm_id"] = catheter_output_admission_level["hadm_id"].astype(str)

cauti_per_admission_full = cauti_per_admission_full.merge(
    catheter_output_admission_level,
    on=["subject_id", "hadm_id"],
    how="left"
)

# 6. Fill NaNs for admissions without any catheter-related outputevents
cauti_per_admission_full["catheter_outputevents"] = cauti_per_admission_full.get("catheter_outputevents", pd.Series([{}]*len(cauti_per_admission_full))).apply(lambda x: x if isinstance(x, dict) else {})
cauti_per_admission_full["catheter_used_in_output_events"] = cauti_per_admission_full.get("catheter_used_in_output_events", pd.Series([[]]*len(cauti_per_admission_full))).apply(lambda x: x if isinstance(x, list) else [])

In [8]:
cauti_per_admission_full

,subject_id,hadm_id,cauti_type,cauti_icd_codes,other_uti_icd_codes,remaining_icd_codes,gender,anchor_age,admittime,dischtime,...,admission_location,discharge_location,race,catheter_procedures,catheter_procedures_ir,catheter_used_in_procedures_ir,catheter_insertion_date_ir,catheter_removal_date_ir,catheter_outputevents,catheter_used_in_output_events
0,10000032,22595853,No_CAUTI,{},{},"{'5723': 'Portal hypertension', '78959': 'Othe...",F,52,2180-05-06 22:23:00,2180-05-07 17:15:00,...,TRANSFER FROM HOSPITAL,HOME,WHITE,{},{},[],NaT,NaT,{},[]
1,10000032,25742920,No_CAUTI,{},{},{'07054': 'Chronic hepatitis C without mention...,F,52,2180-08-05 23:44:00,2180-08-07 17:50:00,...,EMERGENCY ROOM,HOSPICE,WHITE,{},{},[],NaT,NaT,{},[]
2,10000032,29079034,No_CAUTI,{},{},"{'45829': 'Other iatrogenic hypotension', '070...",F,52,2180-07-23 12:35:00,2180-07-25 17:55:00,...,EMERGENCY ROOM,HOME,WHITE,{},{},[],NaT,NaT,{},[]
3,10000560,28979390,No_CAUTI,{},{},"{'1890': 'Malignant neoplasm of kidney, except...",F,53,2189-10-15 10:30:00,2189-10-17 15:00:00,...,PHYSICIAN REFERRAL,HOME,WHITE,{},{},[],NaT,NaT,{},[]
4,10000690,25860671,No_CAUTI,{},{},{'5070': 'Pneumonitis due to inhalation of foo...,F,86,2150-11-02 18:02:00,2150-11-12 13:45:00,...,EMERGENCY ROOM,REHAB,WHITE,{},{},[],NaT,NaT,{226559: 'Foley catheter'},[foley]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158015,19999784,24755486,No_CAUTI,{},{},{'Z5111': 'Encounter for antineoplastic chemot...,M,57,2121-05-30 09:22:00,2121-06-05 08:48:00,...,PHYSICIAN REFERRAL,HOME,BLACK/AFRICAN AMERICAN,{},{},[],NaT,NaT,{},[]
158016,19999784,28216091,No_CAUTI,{},{},{'Z5111': 'Encounter for antineoplastic chemot...,M,57,2119-09-19 10:22:00,2119-09-23 19:00:00,...,PHYSICIAN REFERRAL,HOME HEALTH CARE,BLACK/AFRICAN AMERICAN,{},{},[],NaT,NaT,{},[]
158017,19999828,29734428,Probable_CAUTI,{},"{'N390': 'Urinary tract infection, site not sp...",{'T8131XA': 'Disruption of external operation ...,F,46,2147-07-18 16:23:00,2147-08-04 18:10:00,...,PHYSICIAN REFERRAL,HOME HEALTH CARE,WHITE,{},{},[],NaT,NaT,{},[]
158018,19999840,21033226,No_CAUTI,{},{},"{'3453': 'Grand mal status', '51881': 'Acute r...",M,58,2164-09-10 13:47:00,2164-09-17 13:42:00,...,EMERGENCY ROOM,DIED,WHITE,{},{},[],NaT,NaT,{226559: 'Foley catheter'},[foley]


## Add DateTimeEvents

In [10]:
# -----------------------------
# DATETIMEEVENTS - Only itemid → description in datetimeevents
# AND insertion/removal timestamp columns
# -----------------------------
import pandas as pd
import os

# itemids and mapping
datetime_itemids = [224878, 227584, 229352, 229353]
datetime_item_meta = {
    224878: {"label": "GU Catheter Insertion Date", "catheter_type": "unknown", "role": "insertion"},
    227584: {"label": "GU Catheter D/C Date",        "catheter_type": "unknown", "role": "removal"},
    229352: {"label": "Foley Cath Insertion Date",  "catheter_type": "foley",   "role": "insertion"},
    229353: {"label": "Foley Cath Removal Date",    "catheter_type": "foley",   "role": "removal"},
}

# Load DATETIMEEVENTS
dt = pd.read_csv(
    os.path.join(icu_path, "datetimeevents.csv"),
    usecols=["subject_id", "hadm_id", "itemid", "charttime"],
    dtype={"subject_id": str, "hadm_id": str, "itemid": str},
    low_memory=False
)

# Clean
dt["itemid"] = pd.to_numeric(dt["itemid"], errors="coerce").astype("Int64")
dt = dt.dropna(subset=["itemid"])
dt["itemid"] = dt["itemid"].astype(int)
dt["charttime"] = pd.to_datetime(dt["charttime"], errors="coerce")

# Filter
dt_cat = dt[dt["itemid"].isin(datetime_itemids)].copy()

# Build per-admission result
rows = []
if not dt_cat.empty:
    grouped = dt_cat.groupby(["subject_id", "hadm_id"], sort=False)
    for (subj, hadm), grp in grouped:

        # datetimeevents only contains {itemid : description}
        dt_events_dict = {}

        # track types and timestamps
        types_used = set()
        insertion_times = []
        removal_times = []

        for iid, sub in grp.groupby("itemid"):
            iid = int(iid)
            meta = datetime_item_meta.get(iid, {})
            label = meta.get("label")
            ctype = meta.get("catheter_type")
            role = meta.get("role")

            # Add to dict (NO timestamp)
            dt_events_dict[iid] = label

            if ctype:
                types_used.add(ctype)

            # Earliest timestamp for this itemid
            t = sub["charttime"].min()
            if pd.isna(t):
                continue

            if role == "insertion":
                insertion_times.append(t)
            elif role == "removal":
                removal_times.append(t)

        rows.append({
            "subject_id": str(subj),
            "hadm_id": str(hadm),
            "datetimeevents": dt_events_dict,                                # ← ONLY itemid: description
            "catheter_used_in_datetime_events": sorted(list(types_used)),
            "insertion_date_from_datetimeevents": min(insertion_times) if insertion_times else pd.NaT,
            "removal_date_from_datetimeevents": min(removal_times) if removal_times else pd.NaT
        })

# Build DataFrame even if empty
if rows:
    datetime_admission_level = pd.DataFrame(rows)
else:
    datetime_admission_level = pd.DataFrame(columns=[
        "subject_id","hadm_id","datetimeevents","catheter_used_in_datetime_events",
        "insertion_date_from_datetimeevents","removal_date_from_datetimeevents"
    ])

# Merge into cauti_per_admission_full
for col in ["subject_id","hadm_id"]:
    cauti_per_admission_full[col] = cauti_per_admission_full[col].astype(str)
    datetime_admission_level[col] = datetime_admission_level[col].astype(str)

cauti_per_admission_full = cauti_per_admission_full.merge(
    datetime_admission_level,
    on=["subject_id","hadm_id"],
    how="left"
)

# Fill defaults
cauti_per_admission_full["datetimeevents"] = (
    cauti_per_admission_full["datetimeevents"]
        .apply(lambda x: x if isinstance(x, dict) else {})
)
cauti_per_admission_full["catheter_used_in_datetime_events"] = (
    cauti_per_admission_full["catheter_used_in_datetime_events"]
        .apply(lambda x: x if isinstance(x, list) else [])
)

cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents'],
      dtype='object')

In [11]:
# -----------------------------
# PROCEDURE EVENTS: extract Foley (itemid 229351) into cauti_per_admission_full
# -----------------------------

# candidate filenames in the ICU folder (common variants)
candidate_files = [
    os.path.join(icu_path, "procedureevents.csv")
]

proc_file = None
for f in candidate_files:
    if os.path.exists(f):
        proc_file = f
        break

# If no file found, create an empty DataFrame so rest of code runs safely
if proc_file is None:
    print("No procedureevents file found in icu_path. Creating empty placeholder.")
    proc_df = pd.DataFrame(columns=["subject_id", "hadm_id", "itemid", "starttime"])
else:
    # read minimal columns (read itemid as string then coerce)
    proc_df = pd.read_csv(proc_file, usecols=["subject_id", "hadm_id", "itemid", "starttime"],
                          dtype={"subject_id": str, "hadm_id": str, "itemid": str},
                          low_memory=False)

# normalize itemid -> int and parse charttime (we won't store timestamps in the dict)
proc_df["itemid"] = pd.to_numeric(proc_df["itemid"], errors="coerce").astype("Int64")
proc_df = proc_df.dropna(subset=["itemid"])
proc_df["itemid"] = proc_df["itemid"].astype(int)
# keep charttime parsed if needed for future (not used in dict)
proc_df["starttime"] = pd.to_datetime(proc_df["starttime"], errors="coerce")

# item metadata (only the Foley item requested)
procedure_item_meta = {
    229351: {"label": "Foley Catheter", "catheter_type": "foley"}
}

# Filter to the item(s) of interest
interesting_itemids = set(procedure_item_meta.keys())
proc_cat = proc_df[proc_df["itemid"].isin(interesting_itemids)].copy()

# Build per-admission rows (explicit loop to avoid groupby.apply issues)
rows = []
if not proc_cat.empty:
    grouped = proc_cat.groupby(["subject_id", "hadm_id"], sort=False)
    for (subj, hadm), grp in grouped:
        item_dict = {}
        types_used = set()
        for iid in grp["itemid"].unique():
            iid_int = int(iid)
            meta = procedure_item_meta.get(iid_int, {})
            label = meta.get("label", f"item_{iid_int}")
            ctype = meta.get("catheter_type", None)
            item_dict[iid_int] = label
            if ctype is not None:
                types_used.add(ctype)
        rows.append({
            "subject_id": str(subj),
            "hadm_id": str(hadm),
            "catheter_procedure_events": item_dict,
            "catheter_used_in_procedure_events": sorted(list(types_used))
        })

# Build DataFrame even if empty
if rows:
    procedure_admission_level = pd.DataFrame(rows)
else:
    procedure_admission_level = pd.DataFrame(columns=[
        "subject_id", "hadm_id", "catheter_procedure_events", "catheter_used_in_procedure_events"
    ])

# Align dtypes and merge into cauti_per_admission_full
for col in ["subject_id", "hadm_id"]:
    if col not in cauti_per_admission_full.columns:
        cauti_per_admission_full[col] = ""
    cauti_per_admission_full[col] = cauti_per_admission_full[col].astype(str)
    if col in procedure_admission_level.columns:
        procedure_admission_level[col] = procedure_admission_level[col].astype(str)
    else:
        procedure_admission_level[col] = ""

cauti_per_admission_full = cauti_per_admission_full.merge(
    procedure_admission_level,
    on=["subject_id", "hadm_id"],
    how="left"
)

# Guarantee columns exist and fill defaults
if "catheter_procedure_events" not in cauti_per_admission_full.columns:
    cauti_per_admission_full["catheter_procedure_events"] = [{} for _ in range(len(cauti_per_admission_full))]
else:
    cauti_per_admission_full["catheter_procedure_events"] = cauti_per_admission_full["catheter_procedure_events"].apply(lambda x: x if isinstance(x, dict) else {})

if "catheter_used_in_procedure_events" not in cauti_per_admission_full.columns:
    cauti_per_admission_full["catheter_used_in_procedure_events"] = [[] for _ in range(len(cauti_per_admission_full))]
else:
    cauti_per_admission_full["catheter_used_in_procedure_events"] = cauti_per_admission_full["catheter_used_in_procedure_events"].apply(lambda x: x if isinstance(x, list) else [])

cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events'],
      dtype='object')

In [12]:
# -----------------------------------------------------
# CONFIG
# -----------------------------------------------------
catheter_size_itemid = 224017
chunk_size = 2_000_000

# -----------------------------------------------------
# PATHS (already defined in your notebook)
# -----------------------------------------------------
# icu_path
# cauti_per_admission_full

# -----------------------------------------------------
# STEP 1: Stream-read CHARTEVENTS safely
# -----------------------------------------------------
chunks = []

for chunk in pd.read_csv(
    os.path.join(icu_path, "chartevents.csv"),
    usecols=["subject_id", "hadm_id", "itemid", "charttime", "value"],
    dtype={
        "subject_id": str,
        "hadm_id": str,
        "itemid": str,
        "value": str
    },
    chunksize=chunk_size,
    low_memory=True
):
    # Clean itemid
    chunk["itemid"] = pd.to_numeric(chunk["itemid"], errors="coerce")

    # Filter GU Catheter Size only
    chunk = chunk[chunk["itemid"] == catheter_size_itemid]

    if not chunk.empty:
        chunks.append(chunk)

# -----------------------------------------------------
# STEP 2: Combine filtered chunks
# -----------------------------------------------------
if chunks:
    size_df = pd.concat(chunks, ignore_index=True)
else:
    size_df = pd.DataFrame(
        columns=["subject_id", "hadm_id", "itemid", "charttime", "value"]
    )

# -----------------------------------------------------
# STEP 3: Clean values
# -----------------------------------------------------
size_df["charttime"] = pd.to_datetime(size_df["charttime"], errors="coerce")
size_df = size_df.dropna(subset=["value", "charttime"])

# Standardize value text
size_df["value"] = (
    size_df["value"]
    .astype(str)
    .str.strip()
)

# -----------------------------------------------------
# STEP 4: Admission-level catheter size
# (Earliest documented size)
# -----------------------------------------------------
rows = []

grouped = size_df.groupby(["subject_id", "hadm_id"], sort=False)

for (subject_id, hadm_id), grp in grouped:
    grp = grp.sort_values("charttime")

    earliest_size = grp["value"].iloc[0]

    rows.append({
        "subject_id": str(subject_id),
        "hadm_id": str(hadm_id),
        "catheter_size_from_chartevents": earliest_size
    })

if rows:
    size_admission_level = pd.DataFrame(rows)
else:
    size_admission_level = pd.DataFrame(
        columns=["subject_id", "hadm_id", "catheter_size_from_chartevents"]
    )

# -----------------------------------------------------
# STEP 5: Merge into CAUTI admission-level dataframe
# -----------------------------------------------------
cauti_per_admission_full["subject_id"] = cauti_per_admission_full["subject_id"].astype(str)
cauti_per_admission_full["hadm_id"] = cauti_per_admission_full["hadm_id"].astype(str)

size_admission_level["subject_id"] = size_admission_level["subject_id"].astype(str)
size_admission_level["hadm_id"] = size_admission_level["hadm_id"].astype(str)

cauti_per_admission_full = cauti_per_admission_full.merge(
    size_admission_level,
    on=["subject_id", "hadm_id"],
    how="left"
)

# -----------------------------------------------------
# STEP 6: Fill missing safely
# -----------------------------------------------------
cauti_per_admission_full["catheter_size_from_chartevents"] = (
    cauti_per_admission_full["catheter_size_from_chartevents"]
    .fillna("")
)

# -----------------------------------------------------
# DONE
# -----------------------------------------------------
print("✅ Catheter size extraction completed")
print("Admissions with catheter size:",
      (cauti_per_admission_full["catheter_size_from_chartevents"] != "").sum())


✅ Catheter size extraction completed
Admissions with catheter size: 45612


In [13]:
cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents'],
      dtype='object')

In [14]:
note_path = r"C:\Users\Coditas\Desktop\Projects\Cauti\raw\note"
discharge = pd.read_csv(
    os.path.join(note_path, "discharge.csv"),
    usecols=["subject_id", "hadm_id", "text"]
)

In [15]:
# Each entry: pattern (regex), label (for output), catheter_type (or None)
KEYWORD_PATTERNS = [
    # Foley
    {"pattern": r"\bfoley catheter\b",         "label": "foley catheter",            "type": "foley"},
    {"pattern": r"\bfoley\b",                  "label": "foley",                     "type": "foley"},
    {"pattern": r"\bindwelling foley\b",       "label": "indwelling foley",          "type": "foley"},
    {"pattern": r"\bfoley tubing\b",           "label": "foley tubing",              "type": "foley"},

    # Condom
    {"pattern": r"\bcondom catheter\b",        "label": "condom catheter",           "type": "condom"},
    {"pattern": r"\btexas catheter\b",         "label": "texas catheter",            "type": "condom"},

    # Suprapubic
    {"pattern": r"\bsuprapubic catheter\b",    "label": "suprapubic catheter",       "type": "suprapubic"},
    {"pattern": r"\bsuprapubic tube\b",        "label": "suprapubic tube",           "type": "suprapubic"},
    {"pattern": r"\bsuprapubic\b",             "label": "suprapubic",                "type": "suprapubic"},
    {"pattern": r"\bspc\b",                    "label": "SPC (suprapubic catheter)", "type": "suprapubic"},

    # Straight / intermittent
    {"pattern": r"\bstraight catheter\b",      "label": "straight catheter",         "type": "straight"},
    {"pattern": r"\bstraight cath\b",          "label": "straight cath",             "type": "straight"},
    {"pattern": r"\bintermittent cath\b",      "label": "intermittent cath",         "type": "straight"},
    {"pattern": r"\bintermittent catheter\b",  "label": "intermittent catheter",     "type": "straight"},
    {"pattern": r"\bi&o cath\b",               "label": "I&O cath",                  "type": "straight"},

    # Ureteral
    {"pattern": r"\bureteral catheter\b",      "label": "ureteral catheter",         "type": "ureteral"},
    {"pattern": r"\bureteral stent\b",         "label": "ureteral stent",            "type": "ureteral"},

    # Generic urethral / indwelling urinary catheter
    {"pattern": r"\burethral catheter\b",      "label": "urethral catheter",         "type": "foley"},
    {"pattern": r"\bindwelling urinary catheter\b",
                                              "label": "indwelling urinary catheter","type": "foley"},
    {"pattern": r"\bindwelling catheter\b",    "label": "indwelling catheter",       "type": "foley"},

    # Generic urinary catheter actions (no specific type)
    {"pattern": r"urinary catheter insertion", "label": "urinary catheter insertion","type": None},
    {"pattern": r"urethral catheterization",   "label": "urethral catheterization",  "type": None},
    {"pattern": r"catheter inserted",          "label": "catheter inserted",         "type": None},
    {"pattern": r"catheter placed",            "label": "catheter placed",           "type": None},
    {"pattern": r"catheter placement",         "label": "catheter placement",        "type": None},

    {"pattern": r"foley removed",              "label": "foley removed",             "type": "foley"},
    {"pattern": r"catheter removed",           "label": "catheter removed",          "type": None},
    {"pattern": r"catheter discontinued",      "label": "catheter discontinued",     "type": None},

    # Irrigation / drainage (context, but no specific type)
    {"pattern": r"catheter irrigated",         "label": "catheter irrigated",        "type": None},
    {"pattern": r"bladder irrigation",         "label": "bladder irrigation",        "type": None},
    {"pattern": r"catheter irrigation",        "label": "catheter irrigation",       "type": None},
    {"pattern": r"GU irrigant",                "label": "GU irrigant",               "type": None},

    {"pattern": r"drainage bag",               "label": "drainage bag",              "type": None},
    {"pattern": r"urine drainage bag",         "label": "urine drainage bag",        "type": None},
    {"pattern": r"closed urinary drainage",    "label": "closed urinary drainage",   "type": None},
    {"pattern": r"closed drainage system",     "label": "closed drainage system",    "type": None},
]

# Pre-compile regexes for speed
for kw in KEYWORD_PATTERNS:
    kw["regex"] = re.compile(kw["pattern"], flags=re.IGNORECASE)


In [16]:
def extract_catheter_info_from_text(text: str) -> pd.Series:
    if not isinstance(text, str) or text.strip() == "":
        return pd.Series({
            "catheter_clinical_notes": [],
            "catheter_used_in_clinical_notes": []
        })

    found_labels = set()
    found_types = set()

    for kw in KEYWORD_PATTERNS:
        if kw["regex"].search(text):
            found_labels.add(kw["label"])
            if kw["type"] is not None:
                found_types.add(kw["type"])

    return pd.Series({
        "catheter_clinical_notes": sorted(list(found_labels)),
        "catheter_used_in_clinical_notes": sorted(list(found_types))
    })


In [17]:
def extract_catheter_info_from_text(text: str) -> pd.Series:
    if not isinstance(text, str) or text.strip() == "":
        return pd.Series({
            "catheter_clinical_notes": [],
            "catheter_used_in_clinical_notes": []
        })

    found_labels = set()
    found_types = set()

    for kw in KEYWORD_PATTERNS:
        if kw["regex"].search(text):
            found_labels.add(kw["label"])
            if kw["type"] is not None:
                found_types.add(kw["type"])

    return pd.Series({
        "catheter_clinical_notes": sorted(list(found_labels)),
        "catheter_used_in_clinical_notes": sorted(list(found_types))
    })


In [18]:
# ---------------------------------------------------------
# 1. Ensure merge keys have identical dtypes
# ---------------------------------------------------------
for df in [discharge, cauti_per_admission_full]:
    df["subject_id"] = df["subject_id"].astype("int64")
    df["hadm_id"] = df["hadm_id"].astype("int64")
    
def summarize_discharge_group(group: pd.DataFrame) -> pd.Series:
    combined_text = " ".join(group["text"].dropna().astype(str))
    res = extract_catheter_info_from_text(combined_text)

    return pd.Series({
        "catheter_clinical_notes": res["catheter_clinical_notes"],
        "catheter_used_in_clinical_notes": res["catheter_used_in_clinical_notes"],
    })


catheter_notes_admission_level = (
    discharge
    .groupby(["subject_id", "hadm_id"], as_index=False)
    .apply(summarize_discharge_group, include_groups=False)
    .reset_index()   # <-- CRITICAL
)

cauti_per_admission_full = cauti_per_admission_full.merge(
    catheter_notes_admission_level,
    on=["subject_id", "hadm_id"],
    how="left"
)

# Fill NaNs with clean defaults
cauti_per_admission_full["catheter_clinical_notes"] = cauti_per_admission_full["catheter_clinical_notes"].apply(
    lambda x: x if isinstance(x, list) else []
)
cauti_per_admission_full["catheter_used_in_clinical_notes"] = cauti_per_admission_full["catheter_used_in_clinical_notes"].apply(
    lambda x: x if isinstance(x, list) else []
)
# Keep only admissions where at least one catheter procedure exists
catheter_used_in_clinical_notes = cauti_per_admission_full[
    cauti_per_admission_full["catheter_clinical_notes"].apply(lambda x: isinstance(x, list) and len(x) > 0)
]

In [19]:
cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes'],
      dtype='object')

In [20]:
import pandas as pd

# =====================================================
# Helper
# =====================================================
def has_data(x):
    if isinstance(x, (dict, list)):
        return len(x) > 0
    return False


# =====================================================
# 1. Derive FACTUAL catheter presence (initial)
# =====================================================
def derive_catheter_present(row):

    return (
        has_data(row.get("catheter_procedures")) or
        has_data(row.get("catheter_outputevents")) or
        has_data(row.get("catheter_clinical_notes")) or
        has_data(row.get("datetimeevents")) or
        has_data(row.get("catheter_procedure_events"))
    )


cauti_per_admission_full["catheter_present"] = (
    cauti_per_admission_full.apply(derive_catheter_present, axis=1)
)


# =====================================================
# 2. FORCE catheter_present = True for CAUTI+OtherUTI
# =====================================================
mask_force_catheter = (
    (cauti_per_admission_full["cauti_type"] == "CAUTI+OtherUTI") &
    (cauti_per_admission_full["catheter_present"] == False)
)

cauti_per_admission_full.loc[
    mask_force_catheter,
    "catheter_present"
] = True


# =====================================================
# 3. (Optional) Final CAUTI flag
# =====================================================
def derive_final_cauti_flag(row):

    if row.get("cauti_type") in ("CAUTI", "CAUTI+OtherUTI"):
        return True

    if row.get("cauti_type") == "Probable_CAUTI":
        return row.get("catheter_present") is True

    return False


cauti_per_admission_full["final_cauti_flag"] = (
    cauti_per_admission_full.apply(derive_final_cauti_flag, axis=1)
)

# =====================================================
# 4. SANITY CHECKS (MUST PASS)
# =====================================================
print("\nCAUTI type distribution:")
print(cauti_per_admission_full["cauti_type"].value_counts(dropna=False))

print("\nCatheter present distribution:")
print(cauti_per_admission_full["catheter_present"].value_counts(dropna=False))

print("\nFinal CAUTI flag distribution:")
print(cauti_per_admission_full["final_cauti_flag"].value_counts(dropna=False))



CAUTI type distribution:
cauti_type
No_CAUTI          117182
Probable_CAUTI     39003
CAUTI+OtherUTI      1643
CAUTI                192
Name: count, dtype: int64

Catheter present distribution:
catheter_present
True     138524
False     19496
Name: count, dtype: int64

Final CAUTI flag distribution:
final_cauti_flag
False    136678
True      21342
Name: count, dtype: int64


In [21]:
cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag'],
      dtype='object')

In [22]:
cauti_per_admission_full["cauti_type"].value_counts()

cauti_type
No_CAUTI          117182
Probable_CAUTI     39003
CAUTI+OtherUTI      1643
CAUTI                192
Name: count, dtype: int64

In [23]:
cauti_per_admission_full["catheter_present"].value_counts()

catheter_present
True     138524
False     19496
Name: count, dtype: int64

In [24]:
import pandas as pd

cross_tab = pd.crosstab(
    cauti_per_admission_full["cauti_type"],
    cauti_per_admission_full["catheter_present"]
)

cross_tab


catheter_present,False,True
cauti_type,,
CAUTI,0,192
CAUTI+OtherUTI,0,1643
No_CAUTI,0,117182
Probable_CAUTI,19496,19507


In [26]:
date_cols = [
    "insertion_date_from_datetimeevents",
    "removal_date_from_datetimeevents",
    "catheter_insertion_date_ir",
    "catheter_removal_date_ir"
]

for col in date_cols:
    cauti_per_admission_full[col] = pd.to_datetime(
        cauti_per_admission_full[col],
        errors="coerce"
    )


In [28]:
# Preferred insertion date
cauti_per_admission_full["final_insertion_date"] = (
    cauti_per_admission_full["insertion_date_from_datetimeevents"]
    .combine_first(cauti_per_admission_full["catheter_insertion_date_ir"])
)

# Preferred removal date
cauti_per_admission_full["final_removal_date"] = (
    cauti_per_admission_full["removal_date_from_datetimeevents"]
    .combine_first(cauti_per_admission_full["catheter_removal_date_ir"])
)


In [30]:
cauti_per_admission_full["catheter_duration_days"] = (
    (
        cauti_per_admission_full["final_removal_date"]
        - cauti_per_admission_full["final_insertion_date"]
    )
    .dt.total_seconds() / (24 * 60 * 60)
)


In [32]:
cauti_per_admission_full.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days'],
      dtype='object')

In [34]:
# -----------------------------------------------------
# Save final dataset to Bronze layer
# -----------------------------------------------------

bronze_path = r"C:\Users\Coditas\Desktop\Projects\Cauti\bronze"
os.makedirs(bronze_path, exist_ok=True)

output_file = os.path.join(bronze_path, "bronze_dataset.csv")

cauti_per_admission_full.to_csv(
    output_file,
    index=False
)

print(f"✅ Saved base dataset to: {output_file}")
print(f"Shape: {cauti_per_admission_full.shape}")

✅ Saved base dataset to: C:\Users\Coditas\Desktop\Projects\Cauti\bronze\bronze_dataset.csv
Shape: (158020, 36)
